In [1]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt

In [2]:
from scipy import ndimage

In [3]:
from skimage import measure

In [4]:
def ds9_mask(mask_fits):
    
    # read in mask fits file and initialize WCS 
    hdu_mask = fits.open(mask_fits)
    wcs = WCS(hdu_mask[0].header)
    
    # label each region to masked out and calculate size
    label_im, nb_labels = ndimage.label(hdu_mask[0].data)
    test = measure.regionprops(label_im, hdu_mask[0].data)
    
    # get coords and radius
    radius_array = []
    coords_array = []
    
    for i in range(len(test)):
        radius = hdu_mask[0].header['CDELT2']*np.ceil(test[i].equivalent_diameter)/2
        coords = wcs.wcs_pix2world(test[i].centroid[1], test[i].centroid[0], 0)
        coords = [float(coords[0]), float(coords[1])]
        
        radius_array.append(radius)
        coords_array.append(coords)
        
    # output file that follows ds9 region file standards
    header = """# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
fk5 \n"""
    
    filename = mask_fits.split('.')[0]+'.reg'
    with open(filename, 'w') as file:
        file.write(header)
        for i in range(len(radius_array)):
            line = 'circle('+ str(coords_array[i][0]) + ',' + str(coords_array[i][1]) + ',' + str(radius_array[i]) + ') \n'
            file.write(line)
            
    print('complete ' + mask_fits)

In [5]:
import glob as glob

In [8]:
list_fits = glob.glob("*stars.fits")

In [9]:
for fits_file in list_fits:
    print('starting', fits_file)
    ds9_mask(fits_file)

starting ngc5204_nuv_gauss7p5_stars.fits
complete ngc5204_nuv_gauss7p5_stars.fits
starting ngc4214_nuv_gauss7p5_stars.fits
complete ngc4214_nuv_gauss7p5_stars.fits
starting ngc253_nuv_gauss7p5_stars.fits
complete ngc253_nuv_gauss7p5_stars.fits
starting ngc4449_nuv_gauss7p5_stars.fits
complete ngc4449_nuv_gauss7p5_stars.fits
starting ngc1313_nuv_gauss7p5_stars.fits
complete ngc1313_nuv_gauss7p5_stars.fits
starting ngc625_nuv_gauss7p5_stars.fits
complete ngc625_nuv_gauss7p5_stars.fits
starting ngc7793_nuv_gauss7p5_stars.fits
complete ngc7793_nuv_gauss7p5_stars.fits


In [8]:
# now look at true image with mask overtop and then figure out how to run the photometry again